# Loader for Space Divers Mini sound lib

In [52]:
SOURCE_NAME = "space_divers_mini"
ORIGINAL_PATH = "./data/original/" + SOURCE_NAME + "/"
SAMPLES_PATH = ORIGINAL_PATH + "Samples/"
TARGET_PATH = "./data/intermediate/" + SOURCE_NAME + "/"

## Get the filenames to parse

In [53]:
import os

In [54]:
audio_extensions = (".wav", ".mp3", ".flac", ".aac", ".m4a", ".ogg", ".opus")

def load_filenames(path: str) -> list[str]:
    "Load a list of audio filenames in the directory."
    return [filename for filename in os.listdir(path)
            if filename.endswith(audio_extensions)]

In [55]:
filenames = load_filenames(SAMPLES_PATH)
filenames[:5]

['99S LT Ambience Underwater - Coral Sea.wav',
 '99S LT Manipulate - Distant.wav',
 '99S LT Impact - Crash Distant A.wav',
 '99S LT Underwater - Move D.wav',
 '99S LT Vehicle - Resonation Slow.wav']

In [56]:
extensions = [filename.split(".")[-1] for filename in filenames]
assert len(extensions) == len(filenames)
extensions[:5]

['wav', 'wav', 'wav', 'wav', 'wav']

## Parse the files into labels

The titles follow a couple of patterns but follow within 2:  
`<space separated categories> - <specific label>.wav`  
`<specific label>.wav`  

Space separated categories can also include the `-` split.  
The specific labels can have `A B C...` variants which get ignored.

In [57]:
import string
import stringcase

In [58]:
def parse_filename(filename: str) -> list[str]:
    "Parse the filename to get a list of labels."
    # Remove the prefix and extension
    title = filename.removeprefix("99S LT ").rsplit(".", maxsplit=1)[0]
    *labels, specific_label = title.split("-")
    labels = [l.strip() for label in labels for l in label.split()]
    # Remove the variant at the end
    specific_label = specific_label.rstrip(string.whitespace + string.ascii_uppercase)
    # We have to l.lower() first otherwise stringcase.snakecase will prepend multiple underscores
    normalise = lambda x: stringcase.capitalcase(stringcase.snakecase(x.lower().strip()))
    return [normalise(l) for l in labels + [specific_label]]

In [59]:
labels = [parse_filename(filename) for filename in filenames]
labels[:5]

[['Ambience', 'Underwater', 'Coral_sea'],
 ['Manipulate', 'Distant'],
 ['Impact', 'Crash_distant'],
 ['Underwater', 'Move'],
 ['Vehicle', 'Resonation_slow']]

## Copy the files to the intermediate folder

In [60]:
import xxhash
import shutil

In [61]:
def get_hash(filename: str) -> str:
    "Get the sha2 hash of the file."
    with open(os.path.normpath(filename), "rb") as f:
        return xxhash.xxh64(f.read()).hexdigest()

In [62]:
hashed_filename = ["{}.{}".format(get_hash(SAMPLES_PATH + filename), extension)
                   for filename, extension in zip(filenames, extensions)]
hashed_filename[:5]

['df329b2f45088143.wav',
 'beda557bed5629a2.wav',
 '56b3dae6f6efd75f.wav',
 'baaddb69127f8753.wav',
 '6c110836db6654e5.wav']

In [63]:
if not os.path.exists(TARGET_PATH):
    os.makedirs(TARGET_PATH)

# Copy the files to the target directory
for filename, hash in zip(filenames, hashed_filename):
    shutil.copyfile(SAMPLES_PATH + filename, TARGET_PATH + hash)

## Export the CSV rows

In [64]:
import csv

In [65]:
with open("version", "r") as f:
    version = int(f.read())
print("Version:", version)

Version: 2


In [66]:
# Create a CSV file with the labels
source_rows = [SOURCE_NAME] * len(labels)
version_rows = [version] * len(labels)
with open(TARGET_PATH + "metadata.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "label", "source", "version"])
    writer.writerows(zip(hashed_filename, [",".join(l) for l in labels], source_rows, version_rows))